# Model
Exicting part, try different types regression model.


In [76]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import pickle

import warnings
warnings.filterwarnings('ignore')

In [77]:
#read data, specify index so it's easier to join and search using loc
path = '../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'model_columns_listings.csv')
listings = pd.read_csv(listings_csv)

In [78]:
#shape
listings.shape

(28444, 256)

In [79]:
listings.columns.values

array(['Unnamed: 0', 'id', 'host_since', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_listings_count', 'host_identity_verified', 'latitude',
       'longitude', 'is_location_exact', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'reviews_per_month',
       'last_review_days_ago', 'host_verifications_ facebook',
       'host_verifications_ google', 'host_verifications_ government_id',
       'host_verifications_ identity_manual', 'host_verifications_ jumio',
       'host_verifications_ kba', 'host_verifications_ manual_

In [80]:
#head
listings.head(10)

,Unnamed: 0,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_identity_verified,latitude,...,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,1,2595,733294,3.0,87%,38%,0,6.0,1,40.75362,...,0,0,1,0,0,0,0,0,0,1
1,2,3831,733383,1.0,83%,96%,0,1.0,1,40.68514,...,1,0,0,0,0,0,0,0,0,1
2,3,5099,733440,2.5,NaN,71%,0,1.0,0,40.74767,...,1,0,0,0,0,0,0,0,0,1
3,5,5121,733441,2.5,NaN,67%,0,1.0,0,40.68688,...,0,0,1,0,0,0,0,1,0,0
4,6,5178,733469,3.0,90%,100%,0,1.0,0,40.76489,...,0,0,1,0,0,0,0,0,0,1
5,8,5238,733445,2.0,100%,26%,1,4.0,1,40.71344,...,0,0,1,0,0,0,0,0,0,1
6,9,5441,733454,2.0,100%,56%,1,1.0,1,40.76160,...,0,0,0,0,0,0,0,0,0,1
7,10,5552,733460,3.0,100%,20%,0,1.0,1,40.73552,...,1,0,0,0,0,0,0,0,0,1
8,11,5803,733476,2.0,100%,99%,1,3.0,1,40.66829,...,0,0,1,0,0,0,0,0,0,1
9,12,6021,733493,3.0,90%,84%,0,2.0,0,40.79826,...,0,0,1,0,0,0,0,0,0,1


# Features/Target
Features to be included and test/train split, take out 'latitude', 'longitude'
Also due to removing certain price, the following is gone: 'property_type_Train', 'property_type_Dome house', 'property_type_Dorm', 'host_verifications_google', 'property_type_Island', 'property_type_In-law', 'property_type_Yurt', 'property_type_Bus', 'property_type_Timeshare', 'property_type_Farm stay', 'property_type_Treehouse'
Remove 'review locations', this is the same as neigbourhood

In [81]:

X = listings[['host_since', 'host_response_time', 'host_is_superhost', 'host_listings_count', 
'host_identity_verified', 'is_location_exact', 'accommodates', 'bathrooms', 'bedrooms', 
'beds', 'security_deposit_yes_no', 'cleaning_fee_yes_no', 'guests_included','extra_people_yes_no', 'number_of_reviews', 
'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
'review_scores_checkin', 'review_scores_communication', 'review_scores_value',
'instant_bookable', 'reviews_per_month', 'host_verifications_ facebook', 'host_verifications_ google', 
'host_verifications_ government_id', 'host_verifications_ identity_manual', 'host_verifications_ jumio',
'host_verifications_ kba', 'host_verifications_ manual_offline', 'host_verifications_ manual_online', 
'host_verifications_ offline_government_id', 'host_verifications_ phone', 'host_verifications_ reviews',
'host_verifications_ selfie', 'host_verifications_ sent_id', 'host_verifications_ sesame', 
'host_verifications_ sesame_offline', 'host_verifications_ weibo', 'host_verifications_ work_email', 
'host_verifications_ zhima_selfie', 'host_verifications_email', 'host_verifications_facebook', 
'host_verifications_jumio', 'host_verifications_offline_government_id', 'host_verifications_phone', 
'host_verifications_reviews', 'neighbourhood_group_cleansed_Bronx', 'neighbourhood_group_cleansed_Brooklyn', 
'neighbourhood_group_cleansed_Manhattan', 'neighbourhood_group_cleansed_Queens', 
'neighbourhood_group_cleansed_Staten Island', 'property_type_Apartment','property_type_Barn', 
'property_type_Boat', 'property_type_Bungalow', 'property_type_Cabin', 'property_type_Camper/RV', 
'property_type_Castle', 'property_type_Cave', 'property_type_Condominium', 'property_type_Cottage', 
'property_type_Earth house', 'property_type_Guest suite', 'property_type_Guesthouse', 'property_type_Hostel', 
'property_type_House', 'property_type_Houseboat', 'property_type_Lighthouse','property_type_Loft', 
'property_type_Other', 'property_type_Tent', 'property_type_Tiny house', 
'property_type_Townhouse', 'property_type_Villa', 'room_type_Entire home/apt', 'room_type_Hotel room', 
'room_type_Private room', 'room_type_Shared room', 'cancellation_policy_flexible', 
'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_strict_14_with_grace_period', 
'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'bed_type_Airbed', 'bed_type_Couch', 
'bed_type_Futon','bed_type_Pull-out Sofa', 'bed_type_Real Bed', 'amenities_ toilet', 'amenities_24-hour check-in', 
'amenities_Accessible-height bed','amenities_Accessible-height toilet', 'amenities_Air conditioning', 
'amenities_Air purifier', 'amenities_BBQ grill', 'amenities_Baby bath', 'amenities_Baby monitor',
'amenities_Babysitter recommendations', 'amenities_Baking sheet', 'amenities_Barbecue utensils',
'amenities_Bath towel', 'amenities_Bathrobes', 'amenities_Bathroom essentials', 'amenities_Bathtub',
'amenities_Bathtub with bath chair', 'amenities_Beach essentials', 'amenities_Beachfront', 'amenities_Bed linens', 
'amenities_Bedroom comforts', 'amenities_Bluetooth speaker', 'amenities_Body soap', 'amenities_Bottled water', 
'amenities_Bread maker', 'amenities_Breakfast','amenities_Building staff', 'amenities_Buzzer/wireless intercom',
'amenities_Cable TV', 'amenities_Carbon monoxide detector', 'amenities_Cat(s)', 'amenities_Changing table', 
'amenities_Children’s books and toys', 'amenities_Children’s dinnerware', 'amenities_Cleaning before checkout', 
'amenities_Coffee maker', 'amenities_Cooking basics', 'amenities_Crib', 'amenities_Disabled parking spot',
'amenities_Dishes and silverware', 'amenities_Dishwasher', 'amenities_Dog(s)', 'amenities_Doorman', 'amenities_Dryer',
'amenities_EV charger', 'amenities_Electric profiling bed', 'amenities_Elevator', 'amenities_Essentials', 
'amenities_Ethernet connection', 'amenities_Extra pillows and blankets', 'amenities_Extra space around bed', 
'amenities_Family/kid friendly', 'amenities_Fire extinguisher', 'amenities_Fireplace guards', 
'amenities_Firm mattress', 'amenities_First aid kit', 'amenities_Fixed grab bars for shower',
'amenities_Fixed grab bars for toilet', 'amenities_Flat path to guest entrance',
'amenities_Free parking on premises', 'amenities_Free street parking', 'amenities_Full kitchen', 
'amenities_Game console', 'amenities_Garden or backyard', 'amenities_Ground floor access', 'amenities_Gym',
'amenities_Hair dryer', 'amenities_Handheld shower head', 'amenities_Hangers', 'amenities_Heating', 
'amenities_High chair', 'amenities_Host greets you', 'amenities_Hot tub',
'amenities_Hot water', 'amenities_Hot water kettle', 'amenities_Indoor fireplace', 'amenities_Internet', 
'amenities_Iron', 'amenities_Keypad', 'amenities_Kitchen',
'amenities_Kitchenette', 'amenities_Lake access', 'amenities_Laptop friendly workspace',
'amenities_Lock on bedroom door', 'amenities_Lockbox', 'amenities_Long term stays allowed',
'amenities_Luggage dropoff allowed', 'amenities_Microwave', 'amenities_Mini bar', 'amenities_Mini fridge', 
'amenities_Mobile hoist', 'amenities_No stairs or steps to enter', 'amenities_Other', 'amenities_Other pet(s)',
'amenities_Outlet covers', 'amenities_Oven', 'amenities_Pack ’n Play/travel crib',
'amenities_Paid parking off premises', 'amenities_Paid parking on premises', 'amenities_Patio or balcony', 
'amenities_Pets allowed', 'amenities_Pets live on this property', 'amenities_Pocket wifi', 'amenities_Pool',
'amenities_Private bathroom', 'amenities_Private entrance', 'amenities_Private living room', 
'amenities_Record player', 'amenities_Refrigerator', 'amenities_Room-darkening shades', 'amenities_Safe',
'amenities_Safety card', 'amenities_Self check-in', 'amenities_Shampoo', 'amenities_Shower chair', 
'amenities_Shower gel','amenities_Single level home', 'amenities_Ski-in/Ski-out', 'amenities_Slippers',
'amenities_Smart lock', 'amenities_Smoke detector', 'amenities_Smoking allowed', 'amenities_Snacks',
'amenities_Stair gates', 'amenities_Step-free shower', 'amenities_Stove', 'amenities_Suitable for events',
'amenities_TV', 'amenities_Table corner guards', 'amenities_Toilet paper', 'amenities_Trash can', 
'amenities_Turndown service', 'amenities_Washer', 'amenities_Washer/Dryer', 'amenities_Waterfront',
'amenities_Well-lit path to entrance', 'amenities_Wheelchair accessible', 'amenities_Wide clearance to shower',
'amenities_Wide doorway to guest bathroom', 'amenities_Wide entrance', 'amenities_Wide entrance for guests', 
'amenities_Wide entryway', 'amenities_Wide hallways', 'amenities_Wifi', 'amenities_Window guards',
'last_review_days_ago']]

y = listings['price']

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso Regression

In [82]:
#using CV
regressor = Lasso(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)

print(rmse_scores)


[ 90.41115421  80.15712746  85.45115428  90.33322548  91.73949271
  90.13887667  88.57588157  90.85519288  85.08385955 103.47892828]


In [83]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


89.62248931020292
0.5072202043661358


In [84]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

83.05079890715479
0.5073751644616054


# Ridge Regression

In [85]:
#using CV
regressor = Ridge(alpha=0.5)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)
print(rmse_scores)


[ 89.43799883  79.91434694  85.01279024  89.82545936  91.47681758
  89.92854833  88.38804325  90.55379941  84.33329589 102.71543551]


In [86]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))


89.158653532915
0.5196764839065262


In [87]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

83.07309290237296
0.5071106504546834


# Random Forest

In [88]:
#using CV
regressor = RandomForestRegressor(n_estimators=100, random_state=None, min_samples_split=100)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores) 
print(rmse_scores)

[ 84.69738427  75.7008178   82.9565357   85.04546143  86.25839075
  85.41330232  85.43253988  87.20592286  80.24998739 100.34944466]


In [89]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

85.33097870450588
0.6348028476460521


In [90]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

79.36115582730953
0.5501739084211198


# Gradient Boost

In [91]:
#using CV
regressor = GradientBoostingRegressor(random_state=0)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores) 
print(rmse_scores)

[84.4063651  73.27494879 81.46262852 80.3287669  86.69251742 84.21822006
 83.21160871 83.12772745 78.02181226 97.53368949]


In [92]:
pred_train= regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

83.22782847017481
0.6258150505641243


In [93]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

76.05803025789685
0.5868394739562605


# Feature Importance
this was for gradient boosting - run the same code for random forest, make sure run the fit/model first, it uses same regessor variable

In [94]:
#get features importance
features = regressor.feature_importances_

#get column names from our list of features
column_names=X.columns.values.tolist()

#match features importance to column name
df = pd.DataFrame([features], columns=column_names)


In [95]:
df.head()

,host_since,host_response_time,host_is_superhost,host_listings_count,host_identity_verified,is_location_exact,accommodates,bathrooms,bedrooms,beds,...,amenities_Wheelchair accessible,amenities_Wide clearance to shower,amenities_Wide doorway to guest bathroom,amenities_Wide entrance,amenities_Wide entrance for guests,amenities_Wide entryway,amenities_Wide hallways,amenities_Wifi,amenities_Window guards,last_review_days_ago
0,0.005557,0.000131,0.0,0.008702,0.0,0.0,0.180257,0.165521,0.229509,0.006074,...,0.0,0.0,0.00008,0.0001,0.0,0.000202,0.0,0.0,0.000176,0.002062


In [96]:
#features importance sorted
features_sorted = df.iloc[0].sort_values(ascending=False)

In [97]:
features_sorted.head(60)


bedrooms                                           0.229509
accommodates                                       0.180257
room_type_Entire home/apt                          0.171010
bathrooms                                          0.165521
neighbourhood_group_cleansed_Manhattan             0.118557
property_type_Loft                                 0.011624
amenities_Free street parking                      0.010982
host_listings_count                                0.008702
amenities_Dishwasher                               0.008641
amenities_Patio or balcony                         0.007440
extra_people_yes_no                                0.006630
guests_included                                    0.006245
review_scores_rating                               0.006207
beds                                               0.006074
host_since                                         0.005557
review_scores_cleanliness                          0.005365
amenities_Washer                        

# Model Output for Testing Flask
Using very few features

In [98]:
X = listings[['bedrooms',
            'accommodates',
            'room_type_Entire home/apt',
            'bathrooms',
            'neighbourhood_group_cleansed_Manhattan',
            'property_type_Loft',
            'amenities_Free street parking']]

y = listings['price']

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gradient Boost
For testing flask

In [99]:
#using CV
# regressor = Lasso(alpha=0.1)
# regressor = regressor.fit(X_train, y_train) 

#using CV
regressor = GradientBoostingRegressor(random_state=0)
regressor = regressor.fit(X_train, y_train) 

#cross validation, 10 fold to try next
scores = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10, scoring="neg_mean_squared_error")
rmse_scores = np.sqrt(-scores)

print(rmse_scores)

[86.69528739 77.10785964 86.53339494 84.14168615 89.65546095 87.27550644
 87.11778648 85.00449831 81.1734252  99.76561591]


In [100]:
pred_train = regressor.predict(X_train)
print(rmse_scores.mean())
print(r2_score(y_train, pred_train))

86.44705214009849
0.5685566647172172


In [101]:
pred_test= regressor.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test))) 
print(r2_score(y_test, pred_test))

79.63597311670388
0.5470531366579381


In [102]:
# Saving model to disk
pickle.dump(regressor, open('model.pkl','wb'))